In [2]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'
import meshplot
import trimesh
import kaolin
import open3d as o3d
from glob import glob
from natsort import natsorted
from os.path import join
import numpy as np
import meshplot as mp
import json
# from shape_complete.datagen import get_handcraft_obb 
import seaborn as sns

Warp 1.5.0 initialized:
   CUDA Toolkit 12.6, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:2"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:3"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:4"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:5"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:6"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:7"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
   CUDA peer access:
     Not supported
   Kernel cache:
     /home/ehliang/.cache/warp/1.5.0
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
colors = sns.color_palette('colorblind', 12)
colors = [np.array(c) for c in colors]
def get_lines_from_obb(center, R, extent, rot_matrix=None):
    center, R, extent = [np.array(ls) for ls in (center, R, extent)]
    o3d_obb = o3d.geometry.OrientedBoundingBox(center, R=R, extent=extent)
    if rot_matrix is not None:
        R = rot_matrix[:3,:3]
        trans = rot_matrix[-1, :3]
        o3d_obb.rotate(R)
        # o3d_obb.translate(trans)
    lineset = o3d.geometry.LineSet.create_from_oriented_bounding_box(o3d_obb)
    lines, points = np.array(lineset.lines), np.array(lineset.points)
    start_points = points[lines[:, 0]]
    end_points = points[lines[:, 1]]
    return start_points, end_points
def show_ply_pcds(path, off=1):
    pl = None
    off = np.array([off, 0, 0])
    shading={"point_size": 0.1}
    if not path.endswith("ply"):
        path += "*.ply"
    for idx, fname in enumerate(
        natsorted(glob(path))
    ):
        print(fname)
        pcd = o3d.io.read_point_cloud(fname)
        p = np.array(pcd.points) + idx * off
        if 'sample' in fname:
            # p = p[:12]
            shading={"point_size": 0.1}
        c = np.zeros(p.shape)
        color = colors[min(idx, 11)]
        for i in range(3):
            c[:,i] = color[i]
        if pl is None:
            pl = mp.plot(v=p, return_plot=True, c=c, shading=shading)
        else:
            pl.add_points(p, c=c, shading=shading)
    print(f"Visualized {idx+1} pcds")
    return pl
def show_merged_obj(path, pl=None, offset=0.8, origin=0, smooth=False): 
    off = np.array([offset, 0, 0])
    origin = np.array([0, origin, 0])
    if not path.endswith("obj"):
        path += "*obj"
    print( natsorted(glob(path)))
    for idx, fname in enumerate( natsorted(glob(path)) ) :
        mesh = o3d.io.read_triangle_mesh(fname)
        if smooth: 
            mesh.filter_smooth_laplacian(50, 0.5) 
        v = np.array(mesh.vertices)
        f = np.array(mesh.triangles)
        c = colors[min(idx, 11)]
        if pl is None:
            pl = mp.plot(v=v, f=f, return_plot=True, c=c)
        else:
            pl.add_mesh(v + off*idx + origin, f, c=c)
    # print(f"Visualized {idx} meshes")
    # return pl
def show_obb(path, pl, off=0):
    if not path.endswith(".json"):
        path += "*.json"
    for idx, fname in enumerate(glob(path)):
        with open(fname, "r") as f:
            obb = json.load(f)
        lines = get_lines_from_obb(**obb)
        pl.add_lines(lines[0] + off, lines[1] + off, shading={"line_color": "blue"})

In [4]:
fname = "/store/real/ehliang/r2c2r_blender_data_2/r2c2r_data/test/StorageFurniture/44781/loop_0/link_3.ply"
show_ply_pcds(fname)

/store/real/ehliang/r2c2r_blender_data_2/r2c2r_data/test/StorageFurniture/44781/loop_0/link_3.ply


/store/real/ehliang/.conda/envs/real2code2real/lib/python3.10/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0216998…

Visualized 1 pcds
